<a href="https://colab.research.google.com/github/Abdallahs-Code/regex-automata-tool/blob/main/lexer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import sys
from collections import deque
import json
import itertools
import networkx as nx
import matplotlib.pyplot as plt
from google.colab import files
import json
import sys
from graphviz import Digraph

In [2]:
#Basic NFA structures

class State:
    def __init__(self, name):
        self.name = name
        self.transitions = {}  # key: symbol , value:list of State
        self.is_terminal = False

class NFA:
    def __init__(self, start, end):
        self.start = start      # start and end states
        self.end = end

class RegexToNFA:
    def __init__(self, regex):
        self.regex = regex
        self.state_count = 0

    def new_state(self):
        name = f"S{self.state_count}"
        self.state_count += 1
        return State(name)

    def symbol_nfa(self, symbol):
        start = self.new_state()
        end = self.new_state()
        start.transitions[symbol] = [end]
        return NFA(start, end)

    def concat(self, nfa1, nfa2):
        nfa1.end.transitions.setdefault('ε', []).append(nfa2.start)
        return NFA(nfa1.start, nfa2.end)

    def union(self, nfa1, nfa2):
        start = self.new_state()
        end = self.new_state()
        start.transitions['ε'] = [nfa1.start, nfa2.start]
        nfa1.end.transitions.setdefault('ε', []).append(end)
        nfa2.end.transitions.setdefault('ε', []).append(end)
        return NFA(start, end)

    def star(self, nfa):
        start = self.new_state()
        end = self.new_state()
        start.transitions['ε'] = [nfa.start, end]
        nfa.end.transitions.setdefault('ε', []).extend([start, end])
        return NFA(start, end)

    def plus(self, nfa):
        start = self.new_state()
        end = self.new_state()
        start.transitions['ε'] = [nfa.start]
        nfa.end.transitions.setdefault('ε', []).extend([start, end])
        return NFA(start, end)

    def optional(self, nfa):
        start = self.new_state()
        end = self.new_state()
        start.transitions['ε'] = [nfa.start, end]
        nfa.end.transitions.setdefault('ε', []).append(end)
        return NFA(start, end)

    # old code for tokenizer
    # def tokenize(self):
    #     tokens = []
    #     i = 0
    #     while i < len(self.regex):
    #         c = self.regex[i]
    #         if c == '[':
    #             j = i + 1
    #             charset = set()
    #             while j < len(self.regex) and self.regex[j] != ']':
    #                 # Check that j+1 is '-', j+2 exists, AND j+2 is not ']'
    #                 if (j + 2 < len(self.regex)) and self.regex[j + 1] == '-' and self.regex[j + 2] != ']':
    #                     start_ch, end_ch = self.regex[j], self.regex[j + 2]
    #                     charset.update(chr(k) for k in range(ord(start_ch), ord(end_ch) + 1))
    #                     j += 3
    #                 else:
    #                     charset.add(self.regex[j])
    #                     j += 1
    #             if j >= len(self.regex) or self.regex[j] != ']':
    #                 raise ValueError("Invalid regex: unmatched '['")
    #             tokens.append(charset)
    #             i = j + 1
    #         else:
    #             tokens.append(c)
    #             i += 1
    #     return tokens


    #Tokenizer
    def tokenize(self):
        tokens = []
        i = 0
        while i < len(self.regex):
            c = self.regex[i]

            if c == '[':
                j = i + 1
                square_bracket = c
                while j < len(self.regex) and self.regex[j] != ']':
                    square_bracket += self.regex[j]
                    j += 1
                square_bracket += ']'
                if j >= len(self.regex) or self.regex[j] != ']':
                    raise ValueError("Invalid regex: unmatched '['")
                tokens.append(square_bracket)
                i = j + 1
            else:
                tokens.append(c)
                i += 1
        return tokens


    #Insert concatenation operator (concat) so Shunting-yard algorithm works correctly
    def insert_concat(self, tokens):
        result = []
        for i in range(len(tokens) - 1):
            result.append(tokens[i])
            t1, t2 = tokens[i], tokens[i + 1]
            if ( t1.isalnum() or t1 =='.' or t1 in [')', '*', '+', '?'] or t1.endswith(']')) and\
               ( t2.isalnum() or t2 =='.' or t2 == '(' or t2.startswith('[') ):
                result.append('concat')
        result.append(tokens[-1])
        # print(result)
        return result


    def validate_token(self,token):
        return  token =='.' or(token.isalnum() and token != 'concat')\
            or (token.startswith('[') and token.endswith(']'))

    #Shunting-yard algorithm
    def shunting_yard(self, tokens):
        precedence = {'*': 3, '+': 3, '?': 3, 'concat': 2, '|': 1}
        tokens = self.insert_concat(tokens)
        postfix= []
        stack = []

        for token in tokens:
            if self.validate_token(token):
                postfix.append(token)
            elif token == '(':
                stack.append(token)
            elif token == ')':
                while stack and stack[-1] != '(':
                    postfix.append(stack.pop())
                if not stack:
                    raise ValueError("Mismatched parentheses")
                stack.pop()
            else:     # token is concat or ? or * or + or |
                while stack and stack[-1] != '(' and precedence[stack[-1]] >= precedence[token]:
                    postfix.append(stack.pop())
                stack.append(token)

        while stack:
            if stack[-1] in ['(', ')']:
                raise ValueError("Mismatched parentheses")
            postfix.append(stack.pop())

        return postfix


    #Build NFA
    def build(self):
        tokens = self.tokenize()
        tokens = self.shunting_yard(tokens)
        # print("Postfix:", tokens)
        stack = []

        for token in tokens:
            if self.validate_token(token):
                stack.append(self.symbol_nfa(token))
            elif token == 'concat':
                nfa2 = stack.pop()
                nfa1 = stack.pop()
                stack.append(self.concat(nfa1, nfa2))
            elif token == '|':
                nfa2 = stack.pop()
                nfa1 = stack.pop()
                stack.append(self.union(nfa1, nfa2))
            elif token == '*':
                nfa = stack.pop()
                stack.append(self.star(nfa))
            elif token == '+':
                nfa = stack.pop()
                stack.append(self.plus(nfa))
            elif token == '?':
                nfa = stack.pop()
                stack.append(self.optional(nfa))

        nfa = stack.pop()
        nfa.end.is_terminal = True
        return nfa

#This function performs a full traversal to find all states
def get_all_states(start_state):
        all_states = set()
        queue = deque([start_state])
        visited = set()

        while queue:
            s = queue.popleft()
            if s in visited:
                continue
            visited.add(s)
            all_states.add(s)

            for _, targets in s.transitions.items():
                for t in targets:
                    if t not in visited:
                        queue.append(t)
        return all_states

def renumber_states(nfa):

    queue = deque([nfa.start])
    visited = {nfa.start}
    new_name_counter = 0

    while queue:
        current_state = queue.popleft()
        current_state.name = f"S{new_name_counter}"
        new_name_counter += 1

        for _, targets in current_state.transitions.items():
            for target in targets:
                if target not in visited:
                    visited.add(target)
                    queue.append(target)

#Convert NFA to json file
def nfa_to_json(nfa):
    states = {}
    all_states = get_all_states(nfa.start)
    states["startingState"] = nfa.start.name

    for s in all_states:
        state_info = {"isTerminatingState": s.is_terminal}
        for input, targets in s.transitions.items():
            state_info[input] = [t.name for t in targets]
        states[s.name] = state_info
    return states

In [3]:
def epsilon_closure(nfa, states):
    stack = list(states) # Stack for traversing
    closure = set(states) # Final set to be returned containing the states and all possible epsilon moves from them

    while stack:
        state = stack.pop()
        if "\u03b5" in nfa[state]:
            eps = nfa[state]["\u03b5"]
            if isinstance(eps, str): # If there is only one epsilon going to one state make it a list
                eps = [eps]

            for next_state in eps:
                if next_state not in closure:
                    closure.add(next_state)
                    stack.append(next_state) # Add to be explored so that if this successor state has another epsilon moves

    return closure

def get_input_symbols(nfa):
    # Get all all possible inputs
    symbols = set()

    for state, info in nfa.items():
        if state == "startingState":
            continue

        for key in info:
            if key not in ("isTerminatingState", "\u03b5"):
                symbols.add(key)

    return sorted(symbols)

def convert_nfa_to_dfa(nfa):
    start_state = nfa["startingState"] # Add initial state of the nfa to the initial state of the new dfa
    dfa_start = frozenset(epsilon_closure(nfa, {start_state})) # Add all states by epsilon moves

    queue = [dfa_start] # Add initial state of dfa to the "To be processed queue"
    dfa_states = {dfa_start} # Set to monitor all states

    dfa = {} # Final dfa result
    symbols = get_input_symbols(nfa)

    while queue:
        current = queue.pop(0)
        dfa[current] = {}

        is_accept = any(nfa[s]["isTerminatingState"] for s in current)
        dfa[current]["isTerminatingState"] = is_accept # If any state is an accepting state, set the whole dfa state to final

        for sym in symbols: # Loop on all possible inputs to this new dfa state
            dest = set() # A set (state) that could be new or existing

            for s in current: # Loop on all nfa states and see if they have this input
                if sym in nfa[s]:
                    next_state = nfa[s][sym]
                    if isinstance(next_state, str):
                        next_state = [next_state]

                    for t in next_state:
                        dest.add(t)

            if dest: # We dont want to make an empty state in the dfa
                new_state = frozenset(epsilon_closure(nfa, dest)) # Calculate new dfa state from the input sym to the previous dfa state
                dfa[current][sym] = new_state # Set successor by input sym
                if new_state not in dfa_states: # Dont make new state if it already exists
                    dfa_states.add(new_state)
                    queue.append(new_state)

    return dfa, dfa_start

def dfa_to_json(dfa, start_state):
    # Convert output of the convert_nfa_to_dfa function to a good json format
    result = {}
    state_names = {}

    for i, state in enumerate(dfa.keys()):
        state_names[state] = f"S{i}" # Replacing states in dfa from for example s1,s2,s3 (nfa states) to s0 (dfa state)

    result["startingState"] = state_names[start_state]

    for state, info in dfa.items():
        name = state_names[state]
        result[name] = {}
        result[name]["isTerminatingState"] = info["isTerminatingState"]

        for key, target in info.items():
            if key == "isTerminatingState":
                continue
            result[name][key] = state_names[target]

    return result

In [4]:
# Now the dfa is generated but not minimized

In [5]:
def minimize_dfa(dfa_json):
    states = [s for s in dfa_json if s != "startingState"] # Extracting dfa states
    start_state = dfa_json["startingState"]
    symbols = get_input_symbols(dfa_json) # Collecting symbols

    # Dividing initially onto two groups of accepting and non-accepting states
    accepting = set(s for s in states if dfa_json[s]["isTerminatingState"])
    non_accepting = set(s for s in states if not dfa_json[s]["isTerminatingState"])

    # Handle the case of all states being accepting
    groups = []
    if accepting:
        groups.append(accepting)
    if non_accepting:
        groups.append(non_accepting)

    changed = True # If groups have been divided then iterate again

    while changed:
        changed = False
        new_groups = [] # Future groups for the next iteration

        for group in groups:
            # If group has less than or equal 1 state, then we cannot split
            if len(group) <= 1:
                new_groups.append(group)
                continue # Skip this group
            groups_by_signature = {} # signature -> list of states having the same signature

            for state in group:
                signature = [] # signature: list where i (symbol) maps to element (successor group for this input symbol)

                for sym in symbols:
                    if sym in dfa_json[state]:
                        target = dfa_json[state][sym] # Find which group contains the target state

                        for i, g in enumerate(groups):
                            if target in g:
                                signature.append(i)
                                break
                    else:
                        signature.append(-1) # This state doesnt has the input sym

                signature = tuple(signature)
                if signature not in groups_by_signature:
                    groups_by_signature[signature] = []
                groups_by_signature[signature].append(state) # Append state to other states with the same signature
            if len(groups_by_signature) > 1: # Indicating this group have been splitted
                changed = True # We will try again next iteration

            for g in groups_by_signature.values():
                new_groups.append(set(g)) # Prepare the new splitted groups for the next iteration

        groups = new_groups

    group_names = {} # Naming each group with a single state like s1 s2 s3 -> s0
    result = {} # Building the final result

    # Naming states in the group containing the initial state s0
    start_group_idx = None

    for idx, group in enumerate(groups):
        if start_state in group:

            for state in group:
                group_names[state] = f"S0"
            start_group_idx = idx
            break

    # Naming other states
    current_idx = 1

    for idx, group in enumerate(groups):
        if idx == start_group_idx:
            continue

        for state in group:
            group_names[state] = f"S{current_idx}"
        current_idx += 1

    result["startingState"] = group_names[start_state]

    for idx, group in enumerate(groups):
        rep = next(iter(group)) # Pick a representative to the group
        result[group_names[rep]] = {}
        is_accepting = any(dfa_json[s]["isTerminatingState"] for s in group) # Checking if this state is a final state
        result[group_names[rep]]["isTerminatingState"] = is_accepting

        for sym in symbols:
            if sym in dfa_json[rep]:
                old_target = dfa_json[rep][sym]
                result[group_names[rep]][sym] = group_names[old_target] # Setting the successor state for this input

    return result

In [6]:
def draw_graph(json_file, output_file="nfa_graph"):

    with open(json_file, "r") as f:
        nfa_data = json.load(f)

    starting_state = nfa_data["startingState"]

    dot = Digraph(comment="NFA Graph", format="png")
    dot.attr(rankdir="LR")

    for state_name, info in nfa_data.items():
        if state_name == "startingState":
            continue

        is_terminating = info.get("isTerminatingState", False)
        shape = "doublecircle" if is_terminating else "circle"


        fillcolor = "white"

        dot.node(state_name, state_name, shape=shape, style="filled", fillcolor=fillcolor)


    dot.node("", shape="none", height="0", width="0") # Invisible entry node
    dot.edge("", starting_state, label="Start", color="gray")


    for state_name, info in nfa_data.items():
        if state_name == "startingState":
            continue

        for symbol, target_list in info.items():
            # Skip non-transition keys
            if symbol in ["isTerminatingState"]:
                continue

            if isinstance(target_list, list):
                for target_state in target_list:
                    dot.edge(state_name, target_state, label=str(symbol))
            else:
                dot.edge(state_name, str(target_list), label=str(symbol))

    dot.render(output_file, cleanup=True)

In [7]:
def regex_to_dfa(regex):
  converter = RegexToNFA(regex)
  nfa = converter.build()
  renumber_states(nfa)

  json_output = nfa_to_json(nfa) # Json nfa

  with open("nfa.json", "w") as f: # Json nfa file
      json.dump(json_output, f, indent=2)

  output_name = "nfa_graph"
  draw_graph("nfa.json", output_name) # Nfa graph

  nfa = json_output

  dfa, dfa_start = convert_nfa_to_dfa(nfa)
  dfa_json = dfa_to_json(dfa, dfa_start)

  result = minimize_dfa(dfa_json) # Json dfa

  with open("dfa.json", "w") as f: # Json dfa file
      json.dump(result, f, indent=2)

  output_name = "dfa_graph"
  draw_graph("dfa.json", output_name) # Dfa graph

In [9]:
regex = input("Enter regex: ")
regex_to_dfa(regex)

Enter regex: a*
